In [1]:
import os, re
import codecs
import pymysql
import numpy as np
import pandas as pd

# 连接理赔数据库
conn = pymysql.connect(host='localhost',user = "root",passwd = "gu711205",db='lp_ml_saccvt_not_deduct_cl_tp_fgs',charset = 'gbk')
cursor = conn.cursor()

In [2]:
# 工具函数
def re_FN_month(file_list):
    '''
    筛选包含"月"的文件名
    file_list: 路径下包含的各文件名
    '''
    target_files = []
    for i, name in enumerate(file_list):
        if name.find('月')>=0:
            target_files.append(name)
            
    return target_files

def re_subFN_csv(file_list):
    '''
    筛选包含"月"的子目录下的所有".csv"的文件名
    file_list: 路径下包含的各文件名
    '''
    target_files = []
    for i, name in enumerate(file_list):
        if name.find('.csv')>=0:
            target_files.append(name)
            
    return target_files

def re_subFN_copy_csv(file_list):
    '''
    筛选包含"月"的子目录下的所有"_copy.csv"的文件名
    file_list: 路径下包含的各文件名
    '''
    target_files = []
    for i, name in enumerate(file_list):
        if name.find('_copy.csv')>=0:
            target_files.append(name)
            
    return target_files

def handleEncoding(original_file, newfile):
    f=open(original_file,'rb+')
    content=f.read()#读取文件内容，content为bytes类型，而非string类型
    source_encoding='utf-8'
    #####确定encoding类型
    try:
        content.decode('utf-8').encode('utf-8')
        source_encoding='utf-8'
    except:
        try:
            content.decode('gbk').encode('utf-8')
            source_encoding='gbk'
        except:
            try:
                content.decode('gb2312').encode('utf-8')
                source_encoding='gb2312'
            except:
                try:
                    content.decode('gb18030').encode('utf-8')
                    source_encoding='gb18030'
                except:
                    try:
                        content.decode('big5').encode('utf-8')
                        source_encoding='gb18030'
                    except:
                        content.decode('cp936').encode('utf-8')
                        source_encoding='cp936'
    f.close()
    
    #####按照确定的encoding读取文件内容，并另存为utf-8编码：
    block_size=4096
    
    with codecs.open(original_file,'r',source_encoding) as f:
        with codecs.open(newfile,'w','utf-8') as f2:
            while True:
                content=f.read(block_size)
                if not content:
                    break
                f2.write(content)

def retriveCopyNames(file_names):
    '''
    file_names: list格式，存储同一目录下所有目标文件名称。
    '''
    CopyFile_names = []
    for i in range(len(file_names)):
        tmp = file_names[i][0:file_names[i].rfind('.csv',1)] + '_copy.csv'
        CopyFile_names.append(tmp)
    return CopyFile_names

def save(filename, docs):
    fh = open(filename, 'w', encoding='utf-8')
    for doc in docs:
        fh.write(doc)
        fh.write('\n')
    fh.close()
    
def isNull(values):
    error_type = ['nan', 'NaN', 'NaT']
    if str(values) in error_type:
        output = 'NULL,'
    else:
        output = '\'' + str(values) + '\','
    return output

### 将csv文件重编码为utf-8格式，并将修改后的文件加入_copy.csv后缀：

In [3]:
file_names = os.listdir()
file_names = re_FN_month(file_names)

for i in range(len(file_names)):
    file_path = './' + file_names[i]
    os.chdir(file_path)
    
    file_subnames = os.listdir()
    file_subnames = re_subFN_csv(file_subnames)
    CopyFile_names = retriveCopyNames(file_subnames)
    for i in range(len(file_subnames)):
        handleEncoding(file_subnames[i], CopyFile_names[i])
        
    # 操作完毕后回退上一级路径
    path = os.path.abspath(os.path.dirname(os.getcwd()))
    os.chdir(path) 

### 汇总同一目录下不同月份的数据文件于一张表，并加入ID与DATA_DATE的列表头：

In [4]:
file_names = os.listdir()
file_names = re_FN_month(file_names)
file_merge = pd.DataFrame()
Data_Date = []

for month in file_names:
    file_path = './' + month
    file_subnames = os.listdir(file_path)
    file_subnames = re_subFN_copy_csv(file_subnames)
    
    for k,name in enumerate(file_subnames):
        df = pd.read_csv(file_path + '/'+ name)
        
        Date = re.findall(".*_(.*)_copy.csv.*",file_subnames[k])[0]
        datetime = Date[0:4] + '-' + Date[4:6] + '-' + Date[6:8]
        
        tmp = [datetime] * len(df)
        Data_Date.extend(tmp)
        
        file_merge = file_merge.append(df)
    
    del tmp

file_merge.index = list(range(0, len(file_merge)))
Table_Header = pd.DataFrame({'ID':np.nan * len(df), 'DATA_DATE':Data_Date})
file_merge = pd.concat([Table_Header, file_merge], axis=1)

### 批量生成SQL语句：

In [5]:
# 对于同一目录下不同表，仅需调整table_name唯一的超参数即可
exec_code_ls = []

table_name = 'lp_ml_sasssb_needrecovery_clalist'
cols_len = len(file_merge.iloc[0])

for j in range(len(file_merge)):
    col_values = 'INSERT INTO ' + table_name + ' values('
    # 将"乐观锁",“创建人”,“创建时间”,“更新人”,“更新时间列”设置为空值
    for i in range(cols_len):
        if i == 2:
            col_values += 'NULL,' * 5
        
        col_values += isNull(file_merge.iloc[j][i])
        
    # 去除最后一个逗号
    col_values = col_values[:-1]
    # 加上后缀
    col_values += ');'
    exec_code_ls.append(col_values)
    
# 存储取数入库SQL文件
#save_dir = your_save_file_path
#save(save_dir, exec_code_ls)

### 逐条向数据库中插入SQL语句 (此时库结构已完成设计)：

In [ ]:
for i in range(len(exec_code_ls)):
    sql = exec_code_ls[i]
    try:
       # 执行sql语句
       cursor.execute(sql)
       # 提交到数据库执行
       conn.commit()
    except:
       # 如果发生错误则回滚
       conn.rollback()
        
conn.close()

#### 结果演示:

The kernel below may not be executed

In [6]:
for i in range(2):
    print(exec_code_ls[i])

INSERT INTO lp_ml_sasssb_needrecovery_clalist values(NULL,'2021-02-03',NULL,NULL,NULL,NULL,NULL,'3030100','浙江分公司','杭州中心支公司','电销中心','C330100VEH20040248','F330100A1420038803','王轩','2020-07-13 13:30:05','2020-07-21 11:43:43','无责','3800.0','2021-01-08 10:52:24',NULL,NULL,NULL,NULL,NULL,NULL,'EA1129','王莹','3800.0','追偿申请未发起','否',NULL,NULL,NULL,NULL,'23.54694444444445','否','3800.0','w','20210131','2021-02-01 02:46:58',NULL,'毛灵刚','台州中通星沃汽车销售服务有限公司');
INSERT INTO lp_ml_sasssb_needrecovery_clalist values(NULL,'2021-02-03',NULL,NULL,NULL,NULL,NULL,'3030100','浙江分公司','杭州中心支公司','滨江支公司','C330100VEH20056282','F330100A1420054162','张利斌','2020-09-23 14:30:43','2020-09-23 14:38:28','无责','176000.0','2021-01-11 16:52:20',NULL,NULL,NULL,NULL,NULL,NULL,'EA1129','王莹','176000.0','追偿申请未发起','否',NULL,NULL,NULL,NULL,'20.296990740740743','否','176000.0','w','20210131','2021-02-01 02:46:58',NULL,'姜开楠','浙江捷骏汽车销售服务有限公司9');
